In [36]:
import numpy as np
import pandas as pd
from surprise import Dataset, Reader, BaselineOnly, accuracy, KNNWithMeans
from surprise.model_selection import train_test_split, GridSearchCV

In [2]:
ratings = pd.read_csv('/Users/zanderbonnet/Desktop/GCU/DCS_530/Week 7/Movies/rating.csv')
movies = pd.read_csv('/Users/zanderbonnet/Desktop/GCU/DCS_530/Week 7/Movies/movie.csv')

In [3]:
ratings = ratings.drop('timestamp', axis = 1)
movies['genres'] = movies['genres'].apply(lambda x: x.split('|'))

In [4]:
print(ratings.head(), '\n')
print(movies.head())

   userId  movieId  rating
0       1        2     3.5
1       1       29     3.5
2       1       32     3.5
3       1       47     3.5
4       1       50     3.5 

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                              genres  
0  [Adventure, Animation, Children, Comedy, Fantasy]  
1                     [Adventure, Children, Fantasy]  
2                                  [Comedy, Romance]  
3                           [Comedy, Drama, Romance]  
4                                           [Comedy]  


In [5]:
print(ratings.isnull().sum(), '\n')

print(movies.isnull().sum())

userId     0
movieId    0
rating     0
dtype: int64 

movieId    0
title      0
genres     0
dtype: int64


In [21]:
rating = ratings.sample(round(len(ratings)*.1), random_state=0)

In [22]:
reader = Reader(rating_scale=(1,5))
data = Dataset.load_from_df(rating[['userId', 'movieId', 'rating']], reader)

In [23]:
trainset, testset = train_test_split(data, test_size=0.3, random_state=0)

In [31]:
bsl_options = { "n_epochs": 10, 
              "reg_u": 15, 
              "reg_i": 10}
algo = BaselineOnly(bsl_options = bsl_options)
algo.fit(trainset)

Estimating biases using als...


In [32]:
preds = algo.test(testset)

In [33]:
accuracy.mae(preds)
accuracy.rmse(preds)

MAE:  0.6856
RMSE: 0.8891


0.8891401004998926

In [27]:
bsl_options = { "n_epochs": [5,10], 
              "reg_u": [12,15], 
              "reg_i": [5,10]}

param_grid = {'bsl_options': bsl_options}
gs = GridSearchCV(BaselineOnly, param_grid, measures=["rmse", "mae"], cv=3)
gs.fit(data)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...


In [30]:
print(gs.best_score)
print(gs.best_params["rmse"])

{'rmse': 0.8865748227863531, 'mae': 0.6835004907914187}
{'bsl_options': {'n_epochs': 10, 'reg_u': 12, 'reg_i': 5}}
